<a href="https://colab.research.google.com/github/Kadomium/MSTB/blob/main/MA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df = pd.read_csv('date_raw.csv')
df.head()
#頭2行を削除
df = df.drop([0, 1], axis=0).reset_index(drop=True)
#14行目以降を削除
df = df.iloc[:, :13]
#名前の付け直し
df = df.set_axis(['date', 'yen_usd_open', 'yen_usd_high', 'yen_usd_low', 'yen_usd_last','eur_usd_open', 'eur_usd_high', 'eur_usd_low', 'eur_usd_last','eur_yen_open', 'eur_yen_high', 'eur_yen_low', 'eur_yen_last'],axis='columns')
#数字を文字列ではなく数字と認識してもらう
df[['yen_usd_open', 'yen_usd_high', 'yen_usd_low', 'yen_usd_last','eur_usd_open', 'eur_usd_high', 'eur_usd_low', 'eur_usd_last','eur_yen_open', 'eur_yen_high', 'eur_yen_low', 'eur_yen_last']] = df[['yen_usd_open', 'yen_usd_high', 'yen_usd_low', 'yen_usd_last','eur_usd_open', 'eur_usd_high', 'eur_usd_low', 'eur_usd_last','eur_yen_open', 'eur_yen_high', 'eur_yen_low', 'eur_yen_last']].apply(pd.to_numeric, errors='coerce')
df.head()

,date,yen_usd_open,yen_usd_high,yen_usd_low,yen_usd_last,eur_usd_open,eur_usd_high,eur_usd_low,eur_usd_last,eur_yen_open,eur_yen_high,eur_yen_low,eur_yen_last
0,2000/1/1,102.45,102.59,102.12,102.51,1.0090,1.0136,1.0046,1.0062,103.49,103.51,102.24,103.08
1,2000/1/2,102.45,102.59,102.12,102.51,1.0090,1.0136,1.0046,1.0062,103.49,103.51,102.24,103.08
2,2000/1/3,102.14,102.17,101.40,101.45,1.0073,1.0285,1.0052,1.0243,102.89,104.30,102.17,103.93
3,2000/1/4,101.70,103.30,101.46,103.22,1.0244,1.0341,1.0217,1.0296,104.22,106.56,104.01,106.34
4,2000/1/5,103.17,104.40,102.81,104.14,1.0300,1.0403,1.0285,1.0321,105.77,107.74,105.63,107.63


In [3]:
def get_result(column_name_short, column_name_long, df):
    # パラメータを解析
    n = int(column_name_short.split('_')[-1][2:])  # 'yen_usd_MA5' なら 5 を取得
    m = int(column_name_long.split('_')[-1][2:])  # 'yen_usd_MA50' なら 50 を取得

    # 過去nつ/mつの平均を計算
    ma_short = df['yen_usd_last'].rolling(window=n).mean()
    ma_long = df['yen_usd_last'].rolling(window=m).mean()

    # 1行前のMA値を計算
    prev_ma_short = ma_short.shift(1)
    prev_ma_long = ma_long.shift(1)

   # 2行前のMA値を計算
    prev_ma_short_2 = ma_short.shift(2)
    prev_ma_long_2 = ma_long.shift(2)

    # 売り・買い条件を計算
    sell = (prev_ma_short >= prev_ma_long) & (prev_ma_long_2 >= prev_ma_short_2)
    buy = (prev_ma_long >= prev_ma_short) & (prev_ma_short_2 >= prev_ma_long_2)

    # 売り・買い値を記録
    yen_usd_sell = df['yen_usd_open'][sell].reindex(df.index, fill_value=0)
    yen_usd_buy = df['yen_usd_open'][buy].reindex(df.index, fill_value=0)

    # 利益を計算
    profit = 0
    for i in range(1, len(df)):
        profit = profit - yen_usd_buy.iloc[i] + yen_usd_sell.iloc[i]

    # 最終行で買い越し分を追加
    buy_over = buy.cumsum() - sell.cumsum()
    if buy_over.iloc[-1] > 0:
        profit += df['yen_usd_last'].iloc[-1]

    # 最終行の利益を返す
    return profit

In [6]:
df.head()

,date,yen_usd_open,yen_usd_high,yen_usd_low,yen_usd_last,eur_usd_open,eur_usd_high,eur_usd_low,eur_usd_last,eur_yen_open,eur_yen_high,eur_yen_low,eur_yen_last
0,2000/1/1,102.45,102.59,102.12,102.51,1.0090,1.0136,1.0046,1.0062,103.49,103.51,102.24,103.08
1,2000/1/2,102.45,102.59,102.12,102.51,1.0090,1.0136,1.0046,1.0062,103.49,103.51,102.24,103.08
2,2000/1/3,102.14,102.17,101.40,101.45,1.0073,1.0285,1.0052,1.0243,102.89,104.30,102.17,103.93
3,2000/1/4,101.70,103.30,101.46,103.22,1.0244,1.0341,1.0217,1.0296,104.22,106.56,104.01,106.34
4,2000/1/5,103.17,104.40,102.81,104.14,1.0300,1.0403,1.0285,1.0321,105.77,107.74,105.63,107.63


In [8]:
from joblib import Parallel, delayed

def parallel_get_result(n, m, df):
    return n, m, get_result(f'yen_usd_MA{n}', f'yen_usd_MA{m}', df)

def find_best_combination_parallel(df):
    results = Parallel(n_jobs=-1)(delayed(parallel_get_result)(n, m, df)
                                  for n in range(1, 21)
                                  for m in range(n + 1, 21))
    best_n, best_m, best_profit = max(results, key=lambda x: x[2])
    return best_profit, best_n, best_m


In [9]:
find_best_combination_parallel(df)

(5726.879999999991, 1, 2)